In [1]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import pickle
import matplotlib.pyplot as plt
import gzip

In [5]:
train_data = pd.read_csv("./data/fmnist/fashion-mnist_train.csv")
test_data = pd.read_csv("./data/fmnist/fashion-mnist_test.csv")

In [23]:
train_x = np.array(train_data.iloc[:,1:])
train_y = np.array(train_data.iloc[:, 0])
test_x = np.array(test_data.iloc[:, 1:])
test_y = np.array(test_data.iloc[:,0])
labels_to_name = { 0:"T-shirt/top", 1 :"Trouser", 2 :"Pullover", 3 :"Dress", 4 :"Coat", 5 :"Sandal", 6 :"Shirt", 7 :"Sneaker", 8 :"Bag", 9 :"Ankle boot"}